In [4]:
def get_max_word():
    freq = {}
    for piece in open('example.txt').read().lower().split():
        # only consider alphabetic characters within this piece
        word = ''.join(c for c in piece if c.isalpha())
        if word:
            freq[word] = 1 + freq.get(word, 0)
    max_word = ''
    max_count = 0
    for (w, c) in freq.items():
        if c > max_count:
            max_word = w 
            max_count = c
    print('The mos frequent word is', max_word)
    print('Its number of occurrences is', max_count)

get_max_word()

dict_a = {'name': 'Ann'}
print('name' in dict_a)
# underlying principle

def __contains__(self, k):
    try:
        self[k]                   # access via __getitem___
        return True
    except KeyError:
        return False
    
print(dict_a.setdefault('hobby', ['piano', 'study']))
# underlying principle

def setdefault(self, k, d):
    try:
        return self[k]
    except KeyError:
        self[k] = d
        return d

The mos frequent word is doe
Its number of occurrences is 21
True
['piano', 'study']


In [11]:
from collections.abc import MutableMapping

# MapBase (Abstract Data Type) : Extending the MutableMapping abstract base class
class MapBase(MutableMapping):
    """Our own abstract base class that includes a nonpublic _Item class."""
    # ------------------ nested Item class -----------------------------------

    class _Item:
        """Lightweight composite to store key-value pairs as map items."""
        __slots__ = '_key', '_value'

        def __init__(self, k, v):
            self._key = k 
            self._value = v 

        def __eq__(self, other):
            return self._key == other._key 

        def __ne__(self, other):
            return not (self == other)
    
        def __lt__(self, other):
            return self._key < other._key 

class UnsortedTableMap(MapBase):
    """Map implementation using an unordered list."""
    def __init__(self):
        """Create an empty map."""
        self._table = []
    
    def __getitem__(self, k): # O(n)
        """Return value associated with key k (raise KeyError if not found.)"""
        for item in self._table:
            if k == item._key:
                return item._value 
        raise KeyError('Key Erroro:' + repr(k))
    
    def __setitem__(self, k, v):
        """Assign value v to key k, overwriting existing value if present."""
        for item in self._table:
            if k == item._key:
                item._value = v
                return 
        # did not find match for keyy 
        self._table.append(self._Item(k, v))
    
    def __delitem__(self, k):
        """Remove item associated with key k (raise KeyError if not found)."""
        for j in range(len(self._table)):
            if k == self._table[j].key:
                self._table.pop(j)
                return 
        raise KeyError('Key Error:' + repr(k))

    def __len__(self):
        """Return number of items in the map."""
        return len(self._table)

    def __iter__(self):
        """Generate iteration of the map's keys."""
        for item in self._table:
            yield item._key 

# Practice 
dict_test = UnsortedTableMap()
dict_test['name'] = 'Ann'
dict_test['city'] = 'Taipei'
dict_test['hobby'] = 'piano'
# print(dict_test.get('hobby'))
# print(dict_test.setdefault('height', 160))
# print(len(dict_test))

from random import randrange



class HashMapBase(MapBase):
    """Abstract base class for map using hash-table with MAD compression."""

    def __init__(self, cap=11, p=109345121):
        """Createt an empty hash-table map."""
        self._table = cap * [None]
        self._n = 0                       # number of entries in the map
        self._prime = p                   # prime for MAD compression
        self._scale = 1 + randrange(p-1)  # scale from 1 to p-1 for MAD
        self._shift = randrange(p)        # shift from 0 to p-1 for MAD

    def _hash_function(self, k):
        return (hash(k)*self._scale + self._shift) % self._prime % len(self._table)
    
    def __len__(self):
        return self._n 


    def __getitem__(self, k):
        j = self._hash_function(k)
        return self._bucket_getitem(j, k)        # may raise KeyError
    
    def __setitem__(self, k, v):
        j = self._hash_function(k)
        self._bucket_setitem(j, k, v)
        if self._n > len(self._table) // 2:
            self._resize(2 * len(self._table) - 1)
    
    def __delitem__(self, k):
        j = self._hash_function(k)
        self._bucket_delitem(j, k)
        self._n -= 1
    
    def _resize(self, c):
        old = list(self.items())          # resize bucket array to capacity c
        self._table = c * [None]
        self._n = 0
        for (k, v) in old:
            self[k] = v

# Separate Chaining
class ChainHashMap(HashMapBase):
    """Hash map implemented with separate chaining for collision resolution."""

    def _bucket_getitem(self, j, k):
        bucket = self._table[j]
        if bucket is None:
            raise KeyError('Key Error' + repr(k))        # no match found
        return bucket[k]
    
    def _bucket_setitem(self, j, k, v):
        if self._table[j] is None:
            self._table[j] = UnsortedTableMap()
        oldsize = len(self._table[j])
        self._table[j][k] = v
        if len(self._table[j]) > oldsize:
            self._n += 1
    
    def _bucket_delitem(self, j, k):
        bucket = self._table[j]
        if bucket is None:
            raise KeyError('Key Error' + repr(k))        # no match found
        del bucket[k]
    
    def __iter__(self):
        for bucket in self._table:
            if bucket is not None:
                for key in bucket:
                    yield key 

# Practice 
# ch_map = ChainHashMap()
# ch_map['name'] = 'Ann'
# ch_map['Job'] = 'Software Engineer'
# ch_map['hobby'] = 'piano'
# ch_map["abcd"] = "A"
# ch_map["dcba"] = "B"
# for key,value in ch_map.items():
#     print(key, value)


# Linear Probing
class ProbeHashMap(HashMapBase):
    """Hash map implementated with linear probling for collision resolution."""
    AVAIL = object()           # sentinal marks locations of previous deletions

    def _is_available(self, j):
        """Return True if index j is available in table."""
        return self._table[j] is None or self._table[j] is ProbeHashMap.AVAIL 
    
    def _find_slot(self, j, k):
        """Search for key k in bucket at index j.
        
        Return (success, index) tuple, described as follows:
        If match was found, success is True and index denotes its location.
        If no match found success is False and index denotes first available slot.
        """
        firstAvail = None 
        while True:
            if self._is_available(j):
                if firstAvail is None:
                    firstAvail = j 
                if self._table[j] is None:
                    return (False, firstAvail)          # search has failed
            elif k == self._table[j]._key:
                return (True, j)                        # found a match
            j = (j+1) % len(self._table)
        
    def _bucket_getitem(self, j, k):
        found, s = self._find_slot(j, k)
        if not found:
            raise KeyError('Key Error' + repr(k))        # no match found
        return self._table[s]._value 


    def _bucket_setitem(self, j, k, v):
        found, s = self._find_slot(j, k)
        if not found:
            self._table[s] = self._Item(k, v)             # insert new item
            self._n += 1                                  # size has increased
        else:
            self._table[s]._value = v                     # overwrite existing

    
    def _bucket_delitem(self, j, k):
        found, s = self._find_slot(j, k)
        if not found:
            raise KeyError('Key Error' + repr(k))
        self._table[s] = ProbeHashMap.AVAIL                   # mark as vacated


    def __iter__(self):
        for j in range(len(self._table)):                      # scan entire table
            if not self._is_available(j):
                yield self._table[j]._key


# Practice 
pr_hash = ProbeHashMap()
pr_hash['1'] = 'A'
pr_hash['2'] = 'B'
pr_hash['3'] = 'C'
pr_hash['4'] = 'D'
print(pr_hash.get('4'))
del pr_hash['2']
for k, v in pr_hash.items():
    print(k, v)





D
3 C
1 A
4 D


In [6]:
a = [1, 2, 3]
del a[0]
print(a)

[2, 3]
